In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import pandas as pd
from deap import base, creator, tools
import random

input_file = 'data/TN_NH3_N2O' # 输入数据文件夹
# target = ['TN loss (%)', 'NH3-N loss (%)', 'N2O-N loss (%)']
target = 'N2O-N loss (%)' # 要预测啥就换个名字
data_path = f'{input_file}/data_for_{target}.csv'
# 以LGB举例
model_path = f'output/TN_NH3_N2O/model_{target}/lgb_pred.csv'

In [ ]:
data_all_ef = pd.read_csv(data_path)
X_train = data_all_ef.iloc[:, :-1]
y_train = data_all_ef.iloc[:, -1]

In [ ]:
# 数据集
X = X_train
y = y_train

# 定义优化目标（最小化问题）
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # 创建了一个新的适应度类对象，名为 "FitnessMin"，用于表示一个最小化的优化问题。weights=(-1.0,) 指定了适应度的权重，这里 -1.0 表示我们希望最小化优化目标，因此设置为负数。
creator.create("Individual", list, fitness=creator.FitnessMin)  # fitness=creator.FitnessMin 指定了个体的适应度评价方式，即使用前面定义的最小化的适应度类 "FitnessMin"。

# 初始化遗传算法工具箱
toolbox = base.Toolbox()

# 注册个体和种群的创建函数
toolbox.register("attr_float", random.uniform, 0, 1) # 这行代码注册了一个名为 "attr_float" 的工具，用于生成一个在区间 [0, 1] 内的随机浮点数。

# 这行代码注册了一个名为 "individual" 的工具，用于创建一个个体。tools.initRepeat 是 DEAP 库中的一个函数，用于初始化一个重复的数据结构。
# creator.Individual 指定了要创建的个体的类型，即之前定义的 "Individual" 类。
# toolbox.attr_float 是用于生成个体的基因值的函数，即上面注册的 "attr_float" 工具。
# n=5 指定了个体中基因的数量，这里是5个，对应于需要优化的5个参数。
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=5) 

# 这行代码注册了一个名为 "population" 的工具，用于创建一个种群。list 指定了种群的基本类型，即种群是一个列表（list）。toolbox.individual 是用于生成个体的函数，即上面注册的 "individual" 工具。种群中的每个个体都是通过该函数生成的。
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# 定义评价函数
def evaluate(individual):
    # 根据个体选择相应的特征
    features = [X[:, i] for i, bit in enumerate(individual) if bit > 0.5]
    if len(features) == 0:
        return float('inf'),  # 避免除以零错误
    # 计算随机森林模型的预测值
    '''
    这里需要修改
    '''
    # 这里用随机值代替
    prediction = np.random.rand(100)
    # 计算均方误差（MSE）
    mse = np.mean((prediction - y) ** 2)
    return mse,

# 注册评价函数
toolbox.register("evaluate", evaluate)

# 注册选择、交叉和变异算子
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# 设置遗传算法参数
pop_size = 100
num_generations = 50
cxpb = 0.7
mutpb = 0.2

# 创建初始种群
pop = toolbox.population(n=pop_size)

# 迭代进化
for gen in range(num_generations):
    # 评价种群中的个体
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    # 选择下一代个体
    offspring = toolbox.select(pop, len(pop))
    # 克隆选中的个体
    offspring = list(map(toolbox.clone, offspring))

    # 对选中的个体进行交叉和变异
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < cxpb:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < mutpb:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # 替换当前种群
    pop[:] = offspring

# 找到最优个体
best_ind = tools.selBest(pop, 1)[0]
best_features = [X[:, i] for i, bit in enumerate(best_ind) if bit > 0.5]
print("Best individual:", best_features)
print("MSE:", evaluate(best_ind))
